In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import os

def read_split(dir):
    texts = []
    labels = []
    with open(dir, 'r', encoding='utf-8') as f:
        lines = f.readlines()[1:]
        for line in lines:
            line = line.strip().split(",")
            assert(len(line) == 2)
            label = line[0].strip()
            text = line[1].strip()
            texts.append(text)
            if label == 'AH':
                labels.append(1)
            else:
                labels.append(0)
    return texts, labels


train_texts, train_labels = read_split('/content/gdrive/MyDrive/DL/dataset/pytorch/train.csv')
test_texts, test_labels = read_split('/content/gdrive/MyDrive/DL/dataset/pytorch/test.csv')

In [ ]:
#R = 0.15 # Ratio of training data to be used
#train_texts = train_texts[:int(R * len(train_texts))]
#train_labels = train_labels[:int(R * len(train_labels))]

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
max_seq_length = 64

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, max_length=max_seq_length, padding="max_length")
test_encodings = tokenizer(test_texts, truncation=True, max_length=max_seq_length, padding="max_length")

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)
model.train()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
for epoch in range(3):
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

In [ ]:
import numpy as np
!pip install datasets
from datasets import load_metric

In [ ]:
metric= load_metric("accuracy")
model.eval()
eval_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
for batch in eval_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/gdrive/MyDrive/DL/cnerg-bert-adhominem/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Visualization

## Helper functions

In [ ]:
from transformers import BertModel, BertTokenizer
import re

In [ ]:
model_version = 'utkbert'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

In [ ]:
INTENSITY = 70

def attention_scores(text, layers=None, heads=None):
    sentence_a = text
    inputs = tokenizer.encode_plus(sentence_a, None, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    attention = model(input_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
    sz = len(tokens)
    matrix = [0 for j in range(sz)]
    if layers is None:
        layers = [x for x in range(12)]
    if heads is None:
        heads = [x for x in range(12)]
    for layer in layers:
        for head in heads:
            for j in range(sz):
                matrix[j] += attention[layer][0, head, 0, j].item()
    for j in range(sz):
        matrix[j] = (matrix[j]) / (len(layers) * len(heads))
    return (tokens, matrix)

In [ ]:
def clean_array(w, a):
    W = []
    A = []
    for i in range(len(w)):
        if (w[i].startswith('##')):
            W[len(W) - 1] += w[i][2:]
            A[len(A) - 1] = (A[len(A) - 1] + a[i]) / 2
        else:
            W.append(w[i])
            A.append(a[i])
    return clean_apos(W, A)

def clean_apos(w, a):
    W = []
    A = []
    ctr = 0
    while ctr != len(w):
        if w[ctr] == '\'':
            W[-1] += w[ctr] + w[ctr + 1]
            A[-1] = min(INTENSITY, A[-1] + a[ctr] + a[ctr + 1])
            ctr += 2
        else:
            W.append(w[ctr])
            A.append(a[ctr])
            ctr += 1
    return W, A

In [ ]:
def top_three_tokens(text):
    words, attentions = attention_scores(text)
    words = words[1:-1] # Remove start and end tags
    attentions = attentions[1:-1]
    assert len(words) == len(attentions)
    words, attentions = clean_array(words, attentions)
    assert len(words) == len(attentions)
    top_tokens = list()
    for i in range(len(words)):
        top_tokens.append((attentions[i], i))
    top_tokens = sorted(top_tokens, reverse=True)
    ind = [0]
    cur = 1
    while len(ind) < 3:
        take = True
        for ids in ind:
            take = take and abs(top_tokens[ids][1] - top_tokens[cur][1]) > 2
        if take:
            ind.append(cur)
        cur += 1
    xx = []
    for x in ind:
        xx.append(top_tokens[x][1])
    scores = [0 for i in range(len(words))]
    for w in xx:
        lst = [w - 1, w, w + 1]
        for j in lst:
            if j >= 0 and j < len(words):
                scores[j] = INTENSITY
    return words, scores

In [ ]:
def clean_word(word_list):
  new_word_list = []
  for word in word_list:
    for latex_sensitive in ["\\", "%", "&", "^", "#", "_",  "{", "}"]:
      if latex_sensitive in word:
        word = word.replace(latex_sensitive, '\\'+latex_sensitive)
    new_word_list.append(word)
  return new_word_list

In [ ]:
import string

def sanitize(text):
    text = text.lower()
    text = re.sub("\s+", " ", text)  # converting space-like character to single white space
    text = re.sub("\u2018", '\'', text)    # encoding apostrophe to X
    text = re.sub("\u2019", '\'', text)    # encoding apostrophe to X
    xx = ''
    for x in text:
        if x in string.punctuation and x != '\'':
            xx += ' '
        xx += x
    text = xx
    text = text.split()
    new_text = []
    for x in text:
        ok = False
        for y in x:
            ok = ok or y.isalnum()
        if ok:
            for c in string.punctuation:
                x = x.strip(c)
            new_text.append(x)
    return ' '.join(clean_word(new_text))

In [ ]:
sanitize("'Lol, who ain't you bro??'")

In [ ]:
header = r'''\documentclass[10pt,a4paper]{article}
\usepackage[left=1.00cm, right=1.00cm, top=1.00cm, bottom=2.00cm]{geometry}
\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
\begin{document}
\begin{CJK*}{UTF8}{gbsn}''' + '\n\n'

footer = r'''\end{CJK*}
\end{document}'''

def heatmap(word_list, attention_list, label_list, latex_file, title, batch_size=20, color='blue'):
    '''Routine to generate attention heatmaps for given texts
    ---------------------------------------------------------
    Input:
    :param word_list: array of texts
    :param attention_list: array of attention scores for each text
    :param label_list: label for each text
    :param latex_file: name of the latex file
    :param title: title of latex file
    :param batch_size: Number of comments in each batch
    '''
    with open(latex_file, 'w', encoding='utf-8') as f:
        f.write(header)
        f.write('\\section{%s}\n\n' % title)

        n_examples = len(word_list)
        n_batches = n_examples // batch_size

        for i in range(n_batches):
            batch_word_list = word_list[i * batch_size: (i + 1) * batch_size]
            batch_attention_list = attention_list[i * batch_size: (i + 1) * batch_size]
            batch_label_list = label_list[i * batch_size: (i + 1) * batch_size]
            f.write('\\subsection{Batch %d}\n\n' % (i + 1))
            for j in range(batch_size):
                f.write('\\subsubsection{Comment %d - %s}\n\n' % (j + 1, batch_label_list[j]))
                sentence = batch_word_list[j]
                score = batch_attention_list[j]
                assert len(sentence) == len(score)
                f.write('\\noindent')
                for k in range(len(sentence)):
                    f.write('\\colorbox{%s!%s}{' % (color, score[k]) + '\\strut ' + sentence[k] + '} ')
                f.write('\n\n')

        f.write(footer)

## Visualizing Facebook dataset

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
comments = pickle.load(open('/content/gdrive/MyDrive/DL/Facebook/dataset/classified_comments.pkl', 'rb'))

In [ ]:
comments = sorted(comments, key = lambda x: x['score'])
top_ah_comments = []
top_none_comments = []

for i in range(1000):
    top_ah_comments.append(comments[i])
    top_none_comments.append(comments[-(i + 1)])

In [ ]:
np.random.RandomState(seed=42).shuffle(top_ah_comments)
np.random.RandomState(seed=42).shuffle(top_none_comments)

X = []
Y = []

for x in top_ah_comments:
    c = len(x['text'].strip().split())
    if c >= 20 and c <= 80:
        X.append(x)

for x in top_none_comments:
    c = len(x['text'].strip().split())
    if c >= 20 and c <= 80:
        Y.append(x)

pageX = dict()
pageY = dict()

for x in X:
    try:
        pageX[x['page']].append(x)
    except:
        pageX[x['page']] = []
        pageX[x['page']].append(x)

for y in Y:
    try:
        pageY[y['page']].append(y)
    except:
        pageY[y['page']] = []
        pageY[y['page']].append(y)

In [ ]:
pages = ['DonaldTrump', 'FoxNews', 'Breitbart', 'joebiden', 'barackobama']
groups = []

cnt = dict()

for k in pages:
    cnt[k] = 0

for i in range(5):
    group = []
    for k in pages:
        group.append(pageX[k][cnt[k]])
        group.append(pageY[k][cnt[k]])
        cnt[k] += 1
    for k in pages:
        group.append(pageX[k][cnt[k]])
        group.append(pageY[k][cnt[k]])
        cnt[k] += 1
    np.random.RandomState(seed=i+40).shuffle(group)
    groups.append(group)
    
np.random.RandomState(seed=42).shuffle(groups)

In [ ]:
cols = ['Do you think this is an ad-hominem comment?']
for i in range(1, 20):
    cols.append(cols[0] + '.' + str(i))

In [ ]:
addr = '/content/gdrive/MyDrive/DL/Facebook/annotations/batch{}.csv'
labels = []

for i in range(1, 6):
    df = pd.read_csv(addr.format(i))
    for x in cols:
        lbl = df[x].value_counts().idxmax()
        if lbl == 'Yes':
            labels.append('Ad hominem')
        else:
            labels.append('None')

In [ ]:
texts = []
scores = []

for i in range(5):
    for x in groups[i]:
        sent = x['text']
        sent = sanitize(sent)
        texts_, scores_ = top_three_tokens(sent)
        texts.append(texts_)
        scores.append(scores_)

In [ ]:
heatmap(texts, scores, labels, 'fb.tex', 'Facebook Corpus', color='cyan')

## Visualizing Create Debate

In [ ]:
!git clone https://github.com/utkarsh512/CreateDebate-Scraper.git

In [ ]:
%cd CreateDebate-Scraper/src/nested/

In [ ]:
import re
import pickle
from thread import Thread, Comment
import numpy as np
import pandas as pd

In [ ]:
dir = '/content/gdrive/MyDrive/DL/CreateDebate/Politics/{}.log'
comments_with_score = list()

with open(dir.format('comments_with_score'), 'rb') as f:
    comments_with_score = pickle.load(f)

In [ ]:
reader_addr = '/content/gdrive/MyDrive/DL/CreateDebate/Politics/threads.log'
reader = open(reader_addr, 'rb')
threads = list()
e = Thread()

try:
    while True:
        e = pickle.load(reader)
        threads.append(e)
except:
    reader.close()

authors = dict()
tot_comment_cnt = 0
idx = -1

for thread in threads:
    idx += 1
    for key in thread.comments.keys():
        tot_comment_cnt += 1
        comment = thread.comments[key]
        cur_text = comment.body
        cur_author = comment.author
        url = thread.url
        try:
            authors[cur_author].append((cur_text, url, idx))
        except:
            authors[cur_author] = list()
            authors[cur_author].append((cur_text, url, idx))

cur_author_cnt = 0
cur_comment_cnt = 0
tot_author_cnt = len(authors.keys())
comments_with_url = list()

for author in authors.keys():
    cur_author_cnt += 1
    for i in range(len(authors[author])):
        cur_comment_cnt += 1
        text = [authors[author][i][0]]
        url = authors[author][i][1]
        idx = authors[author][i][2]
        comments_with_url.append((url, text, idx))

In [ ]:
idx = np.random.randint(len(comments_with_score))
print(comments_with_score[idx][1][0])
print(comments_with_url[idx][1][0])

In [ ]:
v = list()
for i in range(len(comments_with_score)):
    score = comments_with_score[i][0]
    text = comments_with_score[i][1]
    url = comments_with_url[i][0]
    idx = comments_with_url[i][2]
    v.append((score, text, url, idx))

In [ ]:
v = sorted(v)
top_ah_comments = []
top_none_comments = []

for i in range(1000):
    top_ah_comments.append(v[i])
    top_none_comments.append(v[-(i + 1)])

In [ ]:
# Random shuffle of top comments
np.random.RandomState(seed=42).shuffle(top_ah_comments)
np.random.RandomState(seed=42).shuffle(top_none_comments)

In [ ]:
small_ah_comments = []
medium_ah_comments = []
large_ah_comments = []

small_none_comments = []
medium_none_comments = []
large_none_comments = []

for x in top_ah_comments:
    c = len(x[1][0].strip().split())
    if c < 38:
        small_ah_comments.append(x)
    elif c >= 80:
        large_ah_comments.append(x)
    else:
        medium_ah_comments.append(x)

for x in top_none_comments:
    c = len(x[1][0].strip().split())
    if c < 34:
        small_none_comments.append(x)
    elif c >= 72:
        large_none_comments.append(x)
    else:
        medium_none_comments.append(x)

total_groups_possible = int(len(small_ah_comments) / 5)
total_groups_possible = min(total_groups_possible, int(len(small_none_comments) / 5))
total_groups_possible = min(total_groups_possible, int(len(large_ah_comments) / 2))
total_groups_possible = min(total_groups_possible, int(len(large_none_comments) / 2))
total_groups_possible = min(total_groups_possible, int(len(medium_ah_comments) / 3))
total_groups_possible = min(total_groups_possible, int(len(medium_none_comments) / 3))
print(total_groups_possible)

In [ ]:
groups = []

small_ah_count = 0
small_none_count = 0
medium_ah_count = 0
medium_none_count = 0
large_ah_count = 0
large_none_count = 0

for i in range(96):
    group = []
    for j in range(5):
        group.append(small_ah_comments[small_ah_count])
        small_ah_count += 1
        group.append(small_none_comments[small_none_count])
        small_none_count += 1
    for j in range(3):
        group.append(medium_ah_comments[medium_ah_count])
        medium_ah_count += 1
        group.append(medium_none_comments[medium_none_count])
        medium_none_count += 1
    for j in range(2):
        group.append(large_ah_comments[large_ah_count])
        large_ah_count += 1
        group.append(large_none_comments[large_none_count])
        large_none_count += 1  
    np.random.RandomState(seed=42).shuffle(group)
    groups.append(group)
    
np.random.RandomState(seed=42).shuffle(groups)

In [ ]:
texts = []
scores = []

low, high = 1, 6

for i in range(low, high):
    for x in groups[i]:
        sent = x[1][0].strip()
        sent = sanitize(sent)
        texts_, scores_ = top_three_tokens(sent)
        texts.append(texts_)
        scores.append(scores_)

In [ ]:
labels = []

for i in range(1, 6):
    df = pd.read_csv(f'/content/gdrive/MyDrive/DL/CreateDebate/Politics/annotations/batch{i}.csv')
    for x in cols:
        lbl = df[x].value_counts().idxmax()
        if lbl == 'Yes':
            labels.append('Ad hominem')
        else:
            labels.append('None')

In [ ]:
heatmap(texts, scores, labels, 'cd.tex', 'Create Debate Corpus', color='cyan')